# Benchmark

This notebook provides a straightforward way to compare the PyZX optimization routines against other approaches on a standard set of benchmark circuits.

First we execute the standard set of imports:

In [11]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx
%config InlineBackend.figure_format = 'svg'

The following class is some boilerplate around the simplification routines so that we can more easily print the relevant metrics

In [12]:
class CircuitComparer:
    def __init__(self, dirname, before, after):
        self.fname_before = os.path.join(dirname, before)
        if after:
            self.fname_after = os.path.join(dirname, after)
        else:
            self.fname_after = ""
        self.fname_tpar = ""
        if before.find('before') != -1:
            self.name = before[:-7]
        else:
            self.name = before
        self.has_run = False
    def __str__(self):
        return "CircuitComparer({}, {})".format(self.name, str(self.has_run))
    def __repr__(self):
        return str(self)
    
    def run(self, validate=True):
        if self.has_run: return True
        c = zx.Circuit.load(self.fname_before).to_basic_gates()
        if self.fname_after:
            c_opt = zx.Circuit.load(self.fname_after).to_basic_gates()
            self.t_opt = c_opt.tcount()
        else:
            self.t_opt = '-'
        self.qubits = c.qubits
        if self.fname_tpar:
            c2 = zx.Circuit.load(self.fname_tpar)
            self.tpar = c2.tcount()
        else: self.tpar = "-"
        self.gatecount = len(c.gates)
        self.t_before = c.tcount()
        g = c.to_graph()
        t = time.time()
        g = zx.simplify.teleport_reduce(g)
        self.t_after = zx.tcount(g)
        self.time_simpl = time.time() - t
        t = time.time()
        c_opt = zx.Circuit.from_graph(g).split_phase_gates().to_basic_gates()
        #c_opt = zx.extract.streaming_extract(g).to_basic_gates()
        c_opt = zx.optimize.basic_optimization(c_opt).to_basic_gates()
        self.c_opt = c_opt
        if validate:
            c_id = c.adjoint()
            c_id.add_circuit(c_opt)
            g = c_id.to_graph()
            zx.simplify.full_reduce(g)
            if g.num_vertices() == 2*len(g.inputs):
                self.verified = "Y"
            else: self.verified = "N"
        else: self.verified = "-"
        
        self.extracts = True
        self.time_extr = 0.0
#         try: 
#             c2 = zx.extract.streaming_extract(g,quiet=True)
#             self.time_extr = time.time() - t
#         except Exception:
#             self.extracts = False
#             self.time_extr = -1
        self.has_run = True
        
        return True
    
    def get_output(self, validate=True):
        if not self.has_run:
            self.run(validate)
        s = self.name.ljust(20) + str(self.qubits).rjust(7)
        s += str(self.gatecount).rjust(8) + str(self.t_before).rjust(9) + str(self.t_opt).rjust(7) 
        s += str(self.tpar).rjust(8) + str(self.t_after).rjust(8)
        s += "{:.2f}".format(self.time_simpl).rjust(11)
        #s += "{:.2f}".format(self.time_extr).rjust(12)
        s += "       " + self.verified
        #s += str(self.hcount).rjust(8) + str(self.vcount).rjust(8)
        return s
    
    def save(self):
        if not self.has_run: self.run()
        fname = self.name
        if not fname.endswith('.qc'): fname = fname + "_pyzx.qc"
        else: fname = fname[:-3] + "_pyzx.qc"
        with open("../circuits/optimized/{}".format(fname),'w') as f:
            f.write(self.c_opt.split_phase_gates().to_qc())

Next we define a function that loads in a directory of circuit files. Note that the directory we target has up to 3 versions of each circuit:

* circuit_before   - This is the original circuit with any modifications, taken from the [Github page](https://github.com/njross/optimizer) of [[1]](#NRSCM)
* circuit_after    - This is the circuit produced by the optimization routines of [[1]](#NRSCM).
* circuit_tpar.qc  - This is the circuit produced by the Tpar algorithm [[2]](#Tpar).
  
<a id="NRSCM"></a>
[1] [Nam, Ross, Su, Childs, Maslov - Automated optimization of large quantum circuits with continuous parameters](https://www.nature.com/articles/s41534-018-0072-4)

<a id="Tpar"></a>
[2] [Amy, Maslov, Mosca - Polynomial-time T-depth Optimization of Clifford+T circuits via Matroid Partitioning](https://arxiv.org/abs/1303.2042)

In [13]:
def load_circuits(directory):
    d = directory
    beforefiles = []
    afterfiles = []
    tparfiles = []
    for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('before') != -1:
            beforefiles.append((f,d))
        elif f.find('tpar') != -1:
            tparfiles.append((f,d))
        elif f.find('.qc') != -1 or f.find('.tfc') != -1:
            beforefiles.append((f,d))
        else: afterfiles.append((f,d))
    
    circuits = []
    for f, d in beforefiles:
        if f.find('before') == -1:
            n = os.path.splitext(f)[0]
        else: n = f[:-7]
        for f2,d2 in afterfiles:
            if d!=d2: continue
            if f2.startswith(n):
                c = CircuitComparer(d, f, f2)
                circuits.append(c)
                break
        else:
            c = CircuitComparer(d, f, '')
            circuits.append(c)
        for f2,d2 in tparfiles:
            if d!=d2: continue
            if f2.startswith(n):
                circuits[-1].fname_tpar = os.path.join(d2,f2)
    
    return circuits

dir_fast_circuits = os.path.join('..', 'circuits', 'Fast')
fast_circuits = load_circuits('/Users/rc/Documents/GitHub/OneQ-Optimization2/Fast')

The directory we target contains a subset of all benchmark circuits, chosen for given quick results. The following cell giving benchmark results of these circuits should therefore only take a few seconds to run. For the benchmarks of slower circuits see [below](#slowbench).
The columns have the following meaning:

* `Circuit     ` - The name of the circuit
* `qubits      ` - Amount of qubits in the circuit
* `G-count     ` - Gate count of original circuit
* `T-before    ` - Amount of T-gates in original circuit
* `T-NRSCM     ` - Amount of T-gates in optimised circuit of [[1]](#NRSCM)
* `T-par       ` - Amount of T-gates in optimised circuit of [[2]](#Tpar)
* `T-PyZX      ` - Amount of T-gates in optimised circuit made by PyZX
* `Time-Simp   ` - The time taken for running the simplification routine on the circuit
* `Time-Extract` - The time taken for extracting the circuit after the simplification

Note that not all circuits were present in the papers [[1]](#NRSCM) and [[2]](#Tpar) in which case the relevant columns are empty.

In [14]:
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp","Verified")
for c in fast_circuits:
    print(c.get_output())

Circuit              qubits G-count T-before T-NRSCM  T-par  T-PyZX  Time-Simp Verified


In [15]:
for c in fast_circuits:
    try: c.save()
    except TypeError: continue

<a id="slowbench"></a>
And now we do the benchmark on the slower circuits. Note that this can take up to half an hour to complete.

In [16]:
dir_slow_circuits = os.path.join('..', 'circuits', 'Slow')
slow_circuits = load_circuits(dir_slow_circuits)
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp", "Verified")
for c in slow_circuits:
    print(c.get_output(validate=False))

FileNotFoundError: [Errno 2] No such file or directory: '../circuits/Slow'

In [ ]:
for c in slow_circuits:
    try: c.save()
    except TypeError: continue